In [1]:
import bpy
import json
from math import *

In [2]:
# bpy 초기화
# clear scene
for elem in bpy.data.objects:
    bpy.data.objects.remove(bpy.data.objects[elem.name], do_unlink=True)
# fbx 메인 모델 불러오기
bpy.ops.import_scene.fbx(
    filepath="default2.fbx"
)

ob = bpy.data.objects['Armature']
bpy.ops.object.mode_set(mode='POSE')

FBX Import: start importing default2.fbx
FBX version: 7400
	FBX import: Prepare...
		Done (0.000000 sec)

	FBX import: Templates...
		Done (0.000000 sec)

	FBX import: Nodes...
		Done (0.000000 sec)

	FBX import: Connections...
		Done (0.000000 sec)

	FBX import: Meshes...
		Done (0.531250 sec)

	FBX import: Materials & Textures...
		Done (0.000000 sec)

	FBX import: Cameras & Lamps...
		Done (0.000000 sec)

	FBX import: Objects & Armatures...
		Done (0.031250 sec)

	FBX import: ShapeKeys...
		Done (0.000000 sec)

	FBX import: Animations...
		Done (0.015625 sec)

	FBX import: Assign materials...
		Done (0.000000 sec)

	FBX import: Assign textures...
		Done (0.000000 sec)

	FBX import: Cycles z-offset workaround...
		Done (0.000000 sec)

	Done (0.656250 sec)

Import finished.


{'FINISHED'}

# 3d 모션 제작 함수

In [3]:
# Func 관절 기초값 설정
def defaultSetting(part, facing, angle):
    pbone = ob.pose.bones[part]
    pbone.rotation_mode = 'XYZ'
    pbone.rotation_euler.rotate_axis(facing, angle)
# Func 윗팔 각도 조정
def UpperArmX(part, rotation, frame):
    if "Left" in part:
        pbone = ob.pose.bones[part]
        pbone.rotation_mode = 'XYZ'
        pbone.rotation_euler.rotate_axis("Y", -rotation[1])
        pbone.rotation_euler.rotate_axis("X", rotation[0])
        pbone.keyframe_insert(data_path="rotation_euler", frame = frame)
        pbone.rotation_euler.rotate_axis("X", -rotation[0])
        pbone.rotation_euler.rotate_axis("Y", rotation[1])
    else:
        pbone = ob.pose.bones[part]
        pbone.rotation_mode = 'XYZ'
        pbone.rotation_euler.rotate_axis("Y", rotation[1])
        pbone.rotation_euler.rotate_axis("X", rotation[0])
        pbone.keyframe_insert(data_path="rotation_euler", frame = frame)
        pbone.rotation_euler.rotate_axis("X", -rotation[0])
        pbone.rotation_euler.rotate_axis("Y", -rotation[1])

In [4]:
# Func 윗팔 각도 조정
pbone = ob.pose.bones["LeftForeArm"]
pbone.rotation_mode = 'XYZ'
pbone.rotation_euler.rotate_axis("Y", radians(-45))
pbone = ob.pose.bones["RightForeArm"]
pbone.rotation_mode = 'XYZ'
pbone.rotation_euler.rotate_axis("Y", radians(45))

def LowerArm(part, rotation, frame):
    pbone = ob.pose.bones[part]
    pbone.rotation_mode = 'XYZ'

    pbone.rotation_euler.rotate_axis("X", rotation)
    pbone.keyframe_insert(data_path="rotation_euler", frame = frame)
    pbone.rotation_euler.rotate_axis("X", -rotation)

    # pbone.rotation_euler.rotate_axis("Z", -rotation)
    # pbone.keyframe_insert(data_path="rotation_euler", frame = frame)
    # pbone.rotation_euler.rotate_axis("Z", rotation)

In [5]:
# Func 손가락 각도 조정
def fingerRotate(part, rotation, frame):
    pbone = ob.pose.bones[part]
    pbone.rotation_mode = 'XYZ'
    if "Thumb" in part:
        weight = 1
        if "Left" in part:
            weight = -1
        pbone.rotation_euler.rotate_axis("Z", -rotation * weight)
        pbone.keyframe_insert(data_path="rotation_euler", frame = frame)
        pbone.rotation_euler.rotate_axis("Z", rotation * weight)
    else:
        pbone.rotation_euler.rotate_axis("X", -rotation)
        pbone.keyframe_insert(data_path="rotation_euler", frame = frame)
        pbone.rotation_euler.rotate_axis("X", rotation)

In [6]:
# 팔 기본 각도 세팅
# defaultSetting("LeftArm", "X", radians(90))
# defaultSetting("RightArm", "X", radians(90))

In [7]:
# json을 통한 모델 리깅
with open('./test.json', 'r') as f:
    frames = json.load(f)
last_frame = int([*frames.keys()][-1].replace("frame_", ""))
for fps in range(last_frame):
    # 어깨 - 팔 각도 조정
    # print(frames[f"frame_{fps}"]["left"]["upperArm"][0])
    UpperArmX("LeftArm", frames[f"frame_{fps}"]["left"]["upperArm"], fps)
    UpperArmX("RightArm", frames[f"frame_{fps}"]["right"]["upperArm"], fps)

    # print(degrees(frames[f"frame_{fps}"]["right"]["lowerArm"][0]))
    LowerArm("LeftForeArm", frames[f"frame_{fps}"]["left"]["lowerArm"][0], fps)
    LowerArm("RightForeArm", frames[f"frame_{fps}"]["right"]["lowerArm"][0], fps)

    # 손가락 각도 조정
    for name in ["Thumb", "Index", "Middle", "Ring", "Pinky"]:
        for idx in range(1, 4):
            if "hand" in frames[f"frame_{fps}"]["left"]:
                fingerRotate(f"LeftHand{name}{idx}", -frames[f"frame_{fps}"]["left"]["hand"][name][str(idx)], fps)
            if "hand" in frames[f"frame_{fps}"]["right"]:
                fingerRotate(f"RightHand{name}{idx}", -frames[f"frame_{fps}"]["right"]["hand"][name][str(idx)], fps)

    print(f'{round(fps/last_frame, 3)  * 100}% 진행중...')
print("완료")

0.0% 진행중...
0.3% 진행중...
0.7000000000000001% 진행중...
1.0% 진행중...
1.3% 진행중...
1.7000000000000002% 진행중...
2.0% 진행중...
2.3% 진행중...
2.7% 진행중...
3.0% 진행중...
3.3000000000000003% 진행중...
3.6999999999999997% 진행중...
4.0% 진행중...
4.3% 진행중...
4.7% 진행중...
5.0% 진행중...
5.3% 진행중...
5.6000000000000005% 진행중...
6.0% 진행중...
6.3% 진행중...
6.6000000000000005% 진행중...
7.000000000000001% 진행중...
7.3% 진행중...
7.6% 진행중...
8.0% 진행중...
8.3% 진행중...
8.6% 진행중...
9.0% 진행중...
9.3% 진행중...
9.6% 진행중...
10.0% 진행중...
10.299999999999999% 진행중...
10.6% 진행중...
11.0% 진행중...
11.3% 진행중...
11.600000000000001% 진행중...
12.0% 진행중...
12.3% 진행중...
12.6% 진행중...
13.0% 진행중...
13.3% 진행중...
13.600000000000001% 진행중...
14.000000000000002% 진행중...
14.299999999999999% 진행중...
14.6% 진행중...
15.0% 진행중...
15.299999999999999% 진행중...
15.6% 진행중...
15.9% 진행중...
16.3% 진행중...
16.6% 진행중...
16.900000000000002% 진행중...
17.299999999999997% 진행중...
17.599999999999998% 진행중...
17.9% 진행중...
18.3% 진행중...
18.6% 진행중...
18.9% 진행중...
19.3% 진행중...
19.6% 진행중...
19.900000000000002% 

In [8]:
#fbx 저장
bpy.ops.export_scene.fbx(
    filepath=bpy.path.abspath("test.fbx"),
    use_active_collection=True
)


FBX export starting... 'test.fbx'
FBX export prepare: Wrapping Objects...
	Done (0.000000 sec)

FBX export prepare: Wrapping Data (lamps, cameras, empties)...
	Done (0.000000 sec)

FBX export prepare: Wrapping Meshes...
(bpy.data.armatures['Armature'], 'POSE')
(bpy.data.armatures['Armature'], 'POSE')
	Done (0.000000 sec)

FBX export prepare: Wrapping ShapeKeys...
	Done (0.000000 sec)

FBX export prepare: Wrapping Armatures...
	Done (0.015625 sec)

FBX export prepare: Wrapping World...
	Done (0.000000 sec)

FBX export prepare: Wrapping Materials...
	Done (0.000000 sec)

FBX export prepare: Wrapping Textures...
	Done (0.000000 sec)

FBX export prepare: Wrapping Animations...
	Done (10.921875 sec)

FBX export prepare: Generating templates...
	Done (0.000000 sec)

FBX export prepare: Generating Connections...
	Done (0.000000 sec)

FBX export fetch empties (1)...
	Done (0.000000 sec)

FBX export fetch lamps (0)...
	Done (0.000000 sec)

FBX export fetch cameras (0)...
	Done (0.000000 sec)



{'FINISHED'}